In [ ]:
%matplotlib ipympl

from matplotlib import pyplot as plt
import numpy as np
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets

import optitrack.csv_reader as csv
from optitrack.geometry import *

In [ ]:
filename = "../material/60fps/skeleton.csv"
# filename = "../material/60fps/rigidbody.csv"

### READ CSV AND LOAD DATA

In [ ]:
# Read the file.
take = csv.Take().readCSV(filename)

# Print out some statistics
print("Found rigid bodies:", take.rigid_bodies.keys())

# Process the first rigid body into a set of planes.
bodies = take.rigid_bodies
skeletons = take.skeletons

bones_pos = []
rigid_body_markers_pos = []
markers_pos = []

if len(bodies) > 0:
    for body in bodies: 
        bones = take.rigid_bodies[body]
        n_frames = bones.num_total_frames()
        
        bones_pos.append(bones.positions)
        for marker in bones.rigid_body_markers.values():
            rigid_body_markers_pos.append(marker.positions)
            
        for marker in bones.rigid_body_markers.values():
            markers_pos.append(marker.positions)

if len(skeletons) > 0:
    for body in skeletons: 
        skeleton = take.skeletons[body]
        n_frames = skeleton.bones["Hip"].num_total_frames()
        
        for marker in skeleton.bones.values():
            rigid_body_markers_pos.append(marker.positions)
            
        for marker in skeleton.bone_markers.values():
            markers_pos.append(marker.positions)
        

#points = bones_pos + rigid_body_markers_pos + markers_pos
points = rigid_body_markers_pos

#points = [[0.0,0.0,0.0] if elem is None else elem for elem in markers for markers in points ]
points = [[[0.0, 0.0, 0.0] if frame is None else frame for frame in markers] for markers in points]
np_points = np.array(points)


In [ ]:
x = np_points[:,:,0]
y = np_points[:,:,1]
z = np_points[:,:,2]

lines_map = {
    list(skeleton.bones).index("Head") : [list(skeleton.bones).index("Neck")],
    list(skeleton.bones).index("Neck") : [list(skeleton.bones).index("RShoulder") , list(skeleton.bones).index("LShoulder"), list(skeleton.bones).index("Chest")],
    
    list(skeleton.bones).index("RShoulder") : [list(skeleton.bones).index("RUArm")],
    list(skeleton.bones).index("RUArm") : [list(skeleton.bones).index("RFArm")],
    list(skeleton.bones).index("RFArm") : [list(skeleton.bones).index("RHand")],
    list(skeleton.bones).index("RHand") : [],
    list(skeleton.bones).index("LShoulder") : [list(skeleton.bones).index("LUArm")],
    list(skeleton.bones).index("LUArm") : [list(skeleton.bones).index("LFArm")],
    list(skeleton.bones).index("LFArm") : [list(skeleton.bones).index("LHand")],
    list(skeleton.bones).index("LHand") : [],
    
    list(skeleton.bones).index("Chest") : [list(skeleton.bones).index("Ab")],
    list(skeleton.bones).index("Ab") : [list(skeleton.bones).index("Hip")],
    list(skeleton.bones).index("Hip") : [list(skeleton.bones).index("LThigh"), list(skeleton.bones).index("RThigh")],
    
    list(skeleton.bones).index("RThigh") : [list(skeleton.bones).index("RShin")],
    list(skeleton.bones).index("RShin") : [list(skeleton.bones).index("RFoot")],
    list(skeleton.bones).index("RFoot") : [list(skeleton.bones).index("RToe")],
    list(skeleton.bones).index("RToe") : [],
    list(skeleton.bones).index("LThigh") : [list(skeleton.bones).index("LShin")],
    list(skeleton.bones).index("LShin") : [list(skeleton.bones).index("LFoot")],
    list(skeleton.bones).index("LFoot") : [list(skeleton.bones).index("LToe")],
    list(skeleton.bones).index("LToe") : []
}

### DISPLACE ANIMATION

In [ ]:
# Create a figure and axes for the plots
fig = plt.figure(figsize=(15, 8))  # Adjust the figure size as needed

# Use GridSpec to customize subplot layout
gs = fig.add_gridspec(2, 3, width_ratios=[1, 1, 1], height_ratios=[1, 1])

# First subplot
ax1 = fig.add_subplot(gs[0, 0], projection='3d')
ax1.view_init(elev=105, azim=-90)  
points1, = ax1.plot([], [], [], "o", color="blue")

# Second subplot
ax2 = fig.add_subplot(gs[0, 1], projection='3d')
ax2.view_init(elev=-180, azim=-90)  
points2, = ax2.plot([], [], [], "o", color="blue")

# Third subplot
ax3 = fig.add_subplot(gs[1, 0], projection='3d')  # Spanning across the bottom row
ax3.view_init(elev=120, azim=-180, roll=-90) 
points3, = ax3.plot([], [], [], "o", color="blue")

# Adjust margins and spacing between subplots
plt.subplots_adjust(left=0.01, right=0.99, bottom=0.01, top=0.99, wspace=0.01, hspace=0.01)

# Define update function
def update(t):
    ax1.clear()
    ax1.set_xlim(np.min(x), np.max(x))
    ax1.set_ylim(np.min(y), np.max(y))
    ax1.set_zlim(np.min(z), np.max(z))
    ax1.scatter(x[:, t], y[:, t], z[:, t])
    ax1.tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)

    ax2.clear()
    ax2.set_xlim(np.min(x), np.max(x))
    ax2.set_ylim(np.min(y), np.max(y))
    ax2.set_zlim(np.min(z), np.max(z))
    ax2.scatter(x[:, t], y[:, t], z[:, t])
    ax2.tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)

    ax3.clear()
    ax3.set_xlim(np.min(x), np.max(x))
    ax3.set_ylim(np.min(y), np.max(y))
    ax3.set_zlim(np.min(z), np.max(z))
    ax3.scatter(x[:, t], y[:, t], z[:, t])    
    ax3.tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)
    # ax3.axis('off')
    
    for i in range(len(x[:,t])) :
       for id in lines_map[i] :
           ax1.plot([x[i,t], x[id,t]], [y[i,t], y[id,t]]  , [z[i,t], z[id,t]]  , "--", color="blue")
           ax2.plot([x[i,t], x[id,t]], [y[i,t], y[id,t]]  , [z[i,t], z[id,t]]  , "--", color="blue")
           ax3.plot([x[i,t], x[id,t]], [y[i,t], y[id,t]]  , [z[i,t], z[id,t]]  , "--", color="blue")

# Create slider widget
frame_slider = widgets.IntSlider(value=0, min=0, max=n_frames-1, step=1, description='Frame:')

# Define function to update plot when slider changes
def slider_changed(change):
    update(change.new)

# Link slider and update function
frame_slider.observe(slider_changed, 'value')

# Create play button
play_button = widgets.Play(
    value=0,
    min=0,
    max=n_frames-1,
    step=1,
    description="Play",
    interval=1  # Adjust the interval (in milliseconds) as needed
)

# Link play button and slider
widgets.jslink((play_button, 'value'), (frame_slider, 'value'))

# Display play button and slider
display(widgets.HBox([play_button, frame_slider]))

# Initial plot
update(0)

### SAVE THE ANIMATION 

In [ ]:
# Create animation
# anim = animation.FuncAnimation(fig, update, frames=n_frames, interval=100)

# Save animation as GIF
# anim.save('animations/animation.gif', writer='pillow', fps=60)